In [1]:
from __future__ import print_function,division
import numpy as np
import pandas as pd
import datetime
from Utils.common.custerReadFile import custom_open
from Algorithm.dfp_xgboost import ios_dfp_xg,ios_dfp_p2,ios_dfp_p4
import json
import re
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

entropy_match = re.compile('^.*_entropy$')

def readData(file_, os_sys="ios"):
    """
    read data from huanbei
    :param file_:
    :param os_sys:
    :return:
    """
    with custom_open(file_) as f:
        data = f
        res = []
        for doc in data:
            doc = json.loads(doc)
            a = {}
            diff = datetime.datetime.fromtimestamp(doc['sourceDoc']['timestamp'] / 1000)\
                              - datetime.datetime.fromtimestamp(doc['query']['timestamp'] / 1000)
            a['event_fly'] = abs(diff.total_seconds() / 60)
        if (doc['sourceDoc']['inApp'] == True and doc['sourceDoc']['inApp'] == doc['query']['inApp']):
            a['dfp_type'] = 0
        elif (doc['sourceDoc']['inApp'] == False and doc['sourceDoc']['inApp'] == doc['query']['inApp']):
            a['dfp_type'] = 1
        else:
            a['dfp_type'] = 2

            for (x, y ) in doc["dfpDetail"].items():
                entropy_flg = entropy_match.match(x)
                if entropy_flg:
                    key = y['addEntropy']['level'] + "_entropy"
                    a[key] = float(y['entropy'])
                else:
                    if x == "constants":
                        pass
                    elif x == "client_ts":
                        key = y['addEntropy']['level'] + "_entropy"
                        a[key] = float(y['entropy'])
                    else:
                        a[x] = y
            if doc.get('label_expect') is not None:
                flg = doc.get('label_expect') == "match"
                if flg:
                    a['label'] = 1
                else:
                    a['label'] = 0
            elif doc.get('expect') is not None:
                flg = doc.get('expect') == "match"
                if flg:
                    a['label'] = 1
                else:
                    a['label'] = 0
            else:
                a['label'] = 0
            a['score'] = doc.get('score',0)
            res.append(a)
        df = pd.DataFrame(res)
        df[['slope']] = df[['slope']].astype(float)
        df[['label']] = df[['label']].fillna(0)

        obj_cols = df.select_dtypes(include=[np.object_]).columns.tolist()
        for col in obj_cols:
            dummy_col = pd.get_dummies(df[col])
            df = pd.concat([df, dummy_col], axis=1)

        df.drop(obj_cols, inplace=True, axis=1)


        ts_match = re.compile('^.*ts_diff$')
        ts_amatch = np.vectorize(lambda x: bool(ts_match.match(x)))
        ts_cols = df.columns.values[ts_amatch(df.columns.values)]
        ts_df = df[ts_cols]
        ts_no = df[df.columns.difference(ts_cols)]
        

        min_max_scaler = MinMaxScaler()
        X_scaled = min_max_scaler.fit_transform(ts_df)
        ts_df = pd.DataFrame(X_scaled, columns=ts_df.columns)
        df = pd.concat([ts_no, ts_df], axis=1)

        return df

/Users/chaoxu/.pyenv/versions/anaconda2-4.3.1/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
file_p1 = "/Users/chaoxu/code/local-spark/Data/sample_huanbei/data.json"

In [3]:
df = readData(file_=file_p1)

In [6]:
df1 = df.loc[df.dfp_type ==2]

In [7]:
df1.head()

,baseEntropy,boottime_entropy,client_ts_entropy,device_browser_engine_entropy,device_model,dfp_type,event_fly,ip_entropy,jsid_entropy,label,resolution,score,slope,client_ts_diff,tcpts_diff,ts_diff
0,1.0,13.233077,3.53957,9.414166,0.0,2,48.535883,19.23456,0.0,0,0.0,1.0,0.965773,0.0,0.0,0.0


In [8]:
df1.label.value_counts()

0    1
Name: label, dtype: int64